In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy_ext as npext
import pandas as pd
import pyximport

pyximport.install(language_level=3,
                  setup_args={'include_dirs': [np.get_include(), npext.np.get_include()]})
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, SpectralEmbedding
from sklearn.preprocessing import MinMaxScaler
import model
from model import CalculateFeatures, SelectFeatures
%matplotlib qt


In [2]:
pdf = pd.DataFrame
df = model.load_data('../../data/DATA_3_south.csv')
df.head()

,Magnitude,Latitude,Longitude,Depth,Datetime
0,3.9,16.8023,-94.9130,114.9,2017-01-01 00:54:31
1,3.9,15.2842,-93.1210,100.7,2017-01-01 04:10:39
2,3.9,17.4503,-94.9543,138.3,2017-01-01 04:23:51
3,3.9,16.4077,-93.7075,139.4,2017-01-01 04:26:01
4,4.1,16.7017,-98.8633,27.5,2017-01-01 08:49:23


In [3]:
if model.features_in_cache():
    df_model = None
    features = model.load_features_from_cache()
else:
    df_model = CalculateFeatures(df, 50, trim_features=True)
    features = df_model.features
features.head()

ValueError: Missing column provided to 'parse_dates': 'Datetime'

In [8]:
df_model.features.to_csv("../../model/features_cache/features.csv", index=False)

In [9]:
features.sort_values('nextDMaxMag',inplace=True)

In [10]:
X = features[model.FEATURES]
Y = features[model.TARGETS]
selected_features = SelectFeatures(features=X, targets=Y, corr_threshold=.95, fixed_features=["b"])

X = X[selected_features[0]].to_numpy()
Y = Y.to_numpy()

In [11]:
Y_Scaler = MinMaxScaler()
X_Scaler = MinMaxScaler()
X_norm64 = X_Scaler.fit_transform(X)
Y_norm64 = Y_Scaler.fit_transform(Y)
X_norm32 = X_norm64.astype(np.float32)
Y_norm32 = Y_norm64.astype(np.float32)

In [12]:
# evaluate isomap with logistic regression algorithm for classification
pca = PCA(n_components=2)
pca.fit(X_norm64.T, Y_norm64.T)
pca_components = pca.components_

In [13]:
cmap = cm.get_cmap("viridis")
colors = list(map(cmap, Y_norm64.T[0]))
sizes = np.power(10, Y_norm64.T[0])

In [14]:
%matplotlib qt
plt.scatter(pca_components[0], pca_components[1], s=sizes, color=colors)

In [31]:
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = df_model.features.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()


In [39]:
embedding = Isomap(n_components=2)
X_transformed = embedding.fit_transform(X_norm32, Y_norm32)

%matplotlib qt
fix, ax = plt.subplots()
ax.set_facecolor((0, 0, .420, 0.125))
ax.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

In [183]:
import matplotlib.pyplot as plt
% matplotlib qt

fig, ax = plt.subplots()
num_ticks = np.round((Y_Scaler.data_max_ - Y_Scaler.data_min_) * 10 + 1).astype(int)[0]
labels = np.round(np.linspace(Y_Scaler.data_min_, Y_Scaler.data_max_, num=num_ticks).T[0], 1)
ticks = np.linspace(0, 1, num=num_ticks)[np.isin(labels, np.unique(Y))]
cax = plt.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

cbar = fig.colorbar(cax, ticks=ticks)
cbar.ax.set_yticklabels(np.unique(Y))
plt.plot()

[]

In [ ]:
% matplotlib qt
import seaborn as sns
import matplotlib.pyplot as plt

corrMatrix = features.corr()

sns.heatmap(corrMatrix, annot=True, cmap="bwr")
plt.show()


In [191]:
Embedding = SpectralEmbedding(n_components=2)
Embedding.fit(X_norm32, Y_norm32)

SystemError: gstrf was called with invalid arguments